In [1]:
import sys 
import xarray as xr
import numpy as np
import os
import warnings
import yaml
from random import randint
import math
from datetime import datetime, timedelta
from parcels import FieldSet, Field, VectorField, ParticleSet, JITParticle, ParcelsRandom, Variable

sys.path.append('/scratch/jvalenti/OParcels_runs/Source') 
from OP_functions import * 

In [2]:
#Only for multicore runs
def simple_partition_function(coords, mpi_size=1):
    """A very simple partition function
    that assigns particles to processors
    """
    return np.linspace(0, mpi_size, coords.shape[0], endpoint=False, dtype=np.int32)

In [3]:
config = ['/home/jvalenti/scratch/OParcels_runs/Parcels_outfall/outfall_runs.yaml']
restart = 0
paths = path()
# Suppress FutureWarning messages
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
param = load_config(config)
#Definitions
start = datetime(param['startdate']['year'], param['startdate']['month'], param['startdate']['day']) #Start date
length = param['param']['length'] # Set Time length [days] 
dt = param['param']['dt'] #toggle between - or + to pick backwards or forwards 
N = param['param']['N'] # Name of deploy loc file
dd = param['param']['dd'] #max depth difference z in N
name = param['file']['name'] #name output file
dtp = param['param']['dtp'] #how often particle released in hours
odt = param['param']['odt'] #how often data is recorded
rrr = param['param']['r'] #radious of particle deployment
MFc = param['param']['MFc']

duration = timedelta(days=length)
print(f"The model will run for {duration.days} days, starting at {start}")

The model will run for 1 days, starting at 2019-01-01 00:00:00


In [5]:
lon, lat, z = pandas_deploy(N,MFc,rrr,dd,dtp)
N = len(lat)
print(f"The model will release {N} particle every timestep")

The model will release 345 particle every timestep


In [6]:
daterange = [start+timedelta(days=i) for i in range(length)]
if restart == 0:
    fn =  name +'_'.join(d.strftime('%Y%m%d')+'_1n' for d in [start, start+duration]) + '.zarr'
else:
    fn =  name + '.zarr'
outfile = os.path.join('/scratch/jvalenti/OParcels_runs/Parcels_outfall/results/', fn)

In [7]:
####BUILD FIELDS FOR SIMULATION######
#Fill in the list of variables that you want to use as fields
varlist=['U','V','W']
filenames,variables=filename_set(start,length,varlist)
dimensions = {'lon': 'glamf', 'lat': 'gphif', 'depth': 'depthw','time': 'time_counter'}
field_set=FieldSet.from_nemo(filenames, variables, dimensions, allow_time_extrapolation=True, chunksize='auto')

#Find file names and variable names ###'Diat','Flag'###
varlist=['US','VS','WL','R','T','S','ssh','Bathy','Kz','totdepth','Vol']
filenames,variables=filename_set(start,length,varlist)

#Add Stokes Drift fields
dimensions = {'lon': 'longitude', 'lat': 'latitude', 'time': 'time'}
us = Field.from_netcdf(filenames['US'], variables['US'], dimensions,allow_time_extrapolation=True, chunksize='auto')
vs = Field.from_netcdf(filenames['VS'], variables['VS'], dimensions,allow_time_extrapolation=True, chunksize='auto')
wl = Field.from_netcdf(filenames['WL'], variables['WL'], dimensions,allow_time_extrapolation=True, chunksize='auto')
field_set.add_field(us)
field_set.add_field(vs)
field_set.add_field(wl)
field_set.add_vector_field(VectorField("stokes", us, vs, wl))

#Add Vertical diffusivity coefficient field
dimensions = {'lon': 'glamt', 'lat': 'gphit', 'depth': 'depthw','time': 'time_counter'}
Kz = Field.from_netcdf(filenames['Kz'], variables['Kz'], dimensions,allow_time_extrapolation=True, chunksize='auto')
field_set.add_field(Kz)

#Add fields located at node T
dimensions = {'lon': 'glamt', 'lat': 'gphit', 'depth': 'deptht','time': 'time_counter'}
#Diat = Field.from_netcdf(filenames['Diat'], variables['Diat'], dimensions,allow_time_extrapolation=True)
#Flag = Field.from_netcdf(filenames['Flag'], variables['Flag'], dimensions,allow_time_extrapolation=True)
R = Field.from_netcdf(filenames['R'], variables['R'], dimensions,allow_time_extrapolation=True, chunksize='auto')
S = Field.from_netcdf(filenames['S'], variables['S'], dimensions,allow_time_extrapolation=True, chunksize='auto')
T = Field.from_netcdf(filenames['T'], variables['T'], dimensions,allow_time_extrapolation=True, chunksize='auto')
#field_set.add_field(Diat)
#field_set.add_field(Flag)
field_set.add_field(R)
field_set.add_field(S)
field_set.add_field(T)

#Add Bathymetry 2D field
dimensions = {'lon': 'glamt', 'lat': 'gphit'}
Bth = Field.from_netcdf(filenames['Bathy'], variables['Bathy'], dimensions,allow_time_extrapolation=True, chunksize='auto')
TD = Field.from_netcdf(filenames['totdepth'], variables['totdepth'], dimensions,allow_time_extrapolation=True, chunksize='auto')
field_set.add_field(Bth)
field_set.add_field(TD)

#Add Volume 3D field
dimensions = {'lon': 'glamt', 'lat': 'gphit', 'depth': 'depthw'}
Vol = Field.from_netcdf(filenames['Vol'], variables['Vol'], dimensions,allow_time_extrapolation=True, chunksize='auto')
field_set.add_field(Vol)

#Add SSH 
dimensions = {'lon': 'glamt', 'lat': 'gphit','time': 'time_counter'}
SSH = Field.from_netcdf(filenames['ssh'], variables['ssh'], dimensions,allow_time_extrapolation=True, chunksize='auto')
field_set.add_field(SSH)

In [8]:
class MPParticle(JITParticle):    
    diameter = Variable('diameter', initial = param['particle']['diameter'])
    length = Variable('length', initial = param['particle']['length'])
    Ub = Variable('Ub', initial = param['particle']['Ub'])  #days to have 67% probability of unbeaching
    tau = Variable('tau', initial =  param['particle']['tau']) # track number of particles
    status = Variable('status', initial = 0)
    fratio = Variable('fratio', initial = param['particle']['fratio'])
    vvl_factor = Variable('fact', initial =  1)    
    ws = Variable('ws', initial =  0) 
    wa = Variable('wa', initial =  0) 
    wm = Variable('wm', initial =  0) 
    cellvol = Variable('cellvol', initial =  0) # MF per parcel
    dlat = Variable('dlat', initial =  0) 
    dlon = Variable('dlon', initial =  0) 
    ddepth = Variable('ddepth', initial =  0) 

In [9]:
if restart=='1':
    #name_temp=find_temp(paths['out'])
    #os.system(f"cd {paths['out']} && parcels_convert_npydir_to_netcdf {name_temp}")
    #outfile=newest(paths['out'])
    print('restarting run with '+outfile)
    pset = ParticleSet.from_particlefile(field_set, MPParticle,outfile,restart=True)
    outfile = outfile[:-5]+'restart.zarr'
else:
    if dtp == 0:
        pset = ParticleSet.from_list(field_set, MPParticle, lon=lon, lat=lat, depth=z,time=start+timedelta(hours=odt))
    else:
        pset = ParticleSet.from_list(field_set, MPParticle, lon=lon, lat=lat, depth=z, repeatdt = timedelta(hours=dtp))
#,Buoyancy,Stokes_drift,turb_mix,Displacement,Unbeaching,DeleteParticle

In [10]:
pset.execute([Advection],
                runtime=duration, 
                dt=dt,
                output_file=pset.ParticleFile(name=outfile, outputdt=timedelta(hours=odt)),
                )

INFO: Output files are stored in /scratch/jvalenti/OParcels_runs/Parcels_outfall/results/Outfalls_run20190101_1n_20190102_1n.zarr.
  0%|          | 0/86400.0 [00:00<?, ?it/s]WARNING: ParticleFile chunks are set to (156, 1), but this may lead to a significant slowdown in Parcels when many calls to repeatdt. Consider setting a larger chunk size for your ParticleFile (e.g. chunks=(int(1e4), 1)).
100%|██████████| 86400.0/86400.0 [05:45<00:00, 250.04it/s]


In [13]:
ds = xr.load_dataset(outfile,decode_times=False)

/home/jvalenti/miniforge3/envs/Parcels_24/lib/python3.11/site-packages/xarray/backends/plugins.py:159: RuntimeWarning: 'netcdf4' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)
/home/jvalenti/miniforge3/envs/Parcels_24/lib/python3.11/site-packages/xarray/backends/plugins.py:159: RuntimeWarning: 'scipy' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)
